In [ ]:
# ratings_electronics dataset
# GConvGRU model

import pandas as pd
import numpy as np
import torch
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch.optim import Adam
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:


df = pd.read_csv('ratings_Electronics (1).csv', names=['user_id', 'item_id', 'rating', 'timestamp'])

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

df = df.dropna(subset=['user_id', 'item_id', 'rating', 'timestamp'])

# edges
user_ids = df['user_id'].unique()
item_ids = df['item_id'].unique()

user_id_mapping = {user_id: idx for idx, user_id in enumerate(user_ids)}
item_id_mapping = {item_id: idx + len(user_ids) for idx, item_id in enumerate(item_ids)}

df['user_idx'] = df['user_id'].map(user_id_mapping)
df['item_idx'] = df['item_id'].map(item_id_mapping)

edge_list = pd.DataFrame({
    'source': df['user_idx'],
    'target': df['item_idx'],
    'rating': df['rating'],
    'timestamp': df['timestamp']
})

edge_index = torch.tensor(edge_list[['source', 'target']].values.T, dtype=torch.long)
edge_attr = torch.tensor(edge_list['rating'].values, dtype=torch.float)

num_nodes = len(user_ids) + len(item_ids)
user_features = np.ones((len(user_ids), 10))  
item_features = np.ones((len(item_ids), 10))  
node_features = np.vstack([user_features, item_features])

node_features_tensor = torch.tensor(node_features, dtype=torch.float)

train_data = Data(x=node_features_tensor, edge_index=edge_index, edge_attr=edge_attr)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

val_edge_list = pd.DataFrame({
    'source': val_df['user_idx'],
    'target': val_df['item_idx'],
    'rating': val_df['rating'],
    'timestamp': val_df['timestamp']
})
val_edge_index = torch.tensor(val_edge_list[['source', 'target']].values.T, dtype=torch.long)
val_edge_attr = torch.tensor(val_edge_list['rating'].values, dtype=torch.float)

val_data = Data(x=node_features_tensor, edge_index=val_edge_index, edge_attr=val_edge_attr)

input_dim = node_features_tensor.shape[1] 
hidden_dim = 64
output_dim = 1  

class TemporalGNN(torch.nn.Module):
    def _init_(self, input_dim, hidden_dim, output_dim, K):
        super(TemporalGNN, self)._init_()
        self.conv1 = GConvGRU(in_channels=input_dim, out_channels=hidden_dim, K=K)
        self.conv2 = GConvGRU(in_channels=hidden_dim, out_channels=output_dim, K=K)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        # x = self.conv2(x, edge_index, edge_attr)
        return x

learning_rate = 0.001
epochs = 100
K = 2  
batch_size = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TemporalGNN(input_dim, hidden_dim, output_dim, K)
model.to(device)  
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)

criterion = torch.nn.MSELoss()  

train_loader = DataLoader([train_data], batch_size=batch_size, shuffle=True)
val_loader = DataLoader([val_data], batch_size=batch_size, shuffle=False)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    for batch in train_loader:
        batch = batch.to(device) 

        out = model(batch.x, batch.edge_index, batch.edge_attr)

        print(f"Output shape: {out.shape}")
        print(f"Edge attribute shape: {batch.edge_attr.shape}")

        if out.shape[0] != batch.edge_attr.shape[0]:
            print(f"Shape mismatch! Output: {out.shape[0]}, Edge attribute: {batch.edge_attr.shape[0]}")
            del out, batch
            continue  

        loss = criterion(out.squeeze(), batch.edge_attr)  
        print(f"Loss: {loss.item()}")

        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in val_loader:
            batch = batch.to(device)
            val_out = model(batch.x, batch.edge_index, batch.edge_attr)

            print(f"Validation output shape: {val_out.shape}")
            print(f"Validation edge attribute shape: {batch.edge_attr.shape}")

            if val_out.shape[0] != batch.edge_attr.shape[0]:
                print(f"Validation shape mismatch! Output: {val_out.shape[0]}, Edge attribute: {batch.edge_attr.shape[0]}")
                del val_out, batch
                continue  

            val_loss += criterion(val_out.squeeze(), batch.edge_attr).item()

        val_loss /= len(val_loader)